<a href="https://colab.research.google.com/github/saurabhsingh1411/Deep_Learning_Projects/blob/main/NLP/TOP%20PROJECTS/Text%20generation/Natural_Language_Generation_using_Sequence_Models(Text_Generation).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf 
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding,LSTM,Dense,Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import numpy as np


Tokenize the sentence 

In [28]:
tokenizer=Tokenizer()
sentence='AI is the new electrcity \n AI is my power'
corpus=sentence.lower().split('\n')
tokenizer.fit_on_texts(corpus)

total_words=len(tokenizer.word_index)+1

In [29]:
corpus

['ai is the new electrcity ', ' ai is my power']

In [30]:
total_words

8

In [31]:
tokenizer.word_index

{'ai': 1, 'electrcity': 5, 'is': 2, 'my': 6, 'new': 4, 'power': 7, 'the': 3}

Generating N gram sequence 

In [32]:
input_sequences = [] #training features (x) will be a list

for line in corpus:
	token_list = tokenizer.texts_to_sequences([line])[0] #converts each sentence as its tokenized equivalent
	for i in range(1, len(token_list)):
		n_gram_sequence = token_list[:i+1] #generating n gram sequences
		input_sequences.append(n_gram_sequence) #appending each n gram sequence to the list of our features (xs)



In [7]:
#input_sequences

labels

In [33]:

max_sequence_len = max([len(x) for x in input_sequences]) #calculating the length of the longest sequence
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre')) #pre-pading each value of the input_sequence
xs, labels = input_sequences[:,:-1],input_sequences[:,-1] #creating xs and their labels using numpy slicing
ys = tf.keras.utils.to_categorical(labels, num_classes=total_words) #creating one hot encoding values

#view rawtrainData.py hosted with ❤ by GitHub

In [34]:
labels

array([2, 3, 4, 5, 2, 6, 7], dtype=int32)

In [35]:
xs

array([[0, 0, 0, 1],
       [0, 0, 1, 2],
       [0, 1, 2, 3],
       [1, 2, 3, 4],
       [0, 0, 0, 1],
       [0, 0, 1, 2],
       [0, 1, 2, 6]], dtype=int32)

Constructing the model: Single Layer LSTM Model

In [36]:
model = Sequential() #creating a sequential model
model.add(Embedding(total_words, 64, input_length=max_sequence_len)) #adding an embedding layer with 64 as the embedding dimension
model.add(Bidirectional(LSTM(20))) #adding 20 LSTM units
model.add(Dense(total_words, activation='softmax')) #creating a dense layer with 54 output units (total_words) with softmax activation


In [37]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 5, 64)             512       
_________________________________________________________________
bidirectional_2 (Bidirection (None, 40)                13600     
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 328       
Total params: 14,440
Trainable params: 14,440
Non-trainable params: 0
_________________________________________________________________


In [38]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) #compiling the model with adam optimiser
history = model.fit(xs, ys, epochs=500, verbose=1) #training for 500 epochs

Epoch 1/500
1/1 [==============================] - 4s 4s/step - loss: 2.0838 - accuracy: 0.0000e+00
Epoch 2/500
1/1 [==============================] - 0s 12ms/step - loss: 2.0775 - accuracy: 0.0000e+00
Epoch 3/500
1/1 [==============================] - 0s 15ms/step - loss: 2.0712 - accuracy: 0.2857
Epoch 4/500
1/1 [==============================] - 0s 13ms/step - loss: 2.0649 - accuracy: 0.2857
Epoch 5/500
1/1 [==============================] - 0s 7ms/step - loss: 2.0586 - accuracy: 0.2857
Epoch 6/500
1/1 [==============================] - 0s 10ms/step - loss: 2.0523 - accuracy: 0.2857
Epoch 7/500
1/1 [==============================] - 0s 11ms/step - loss: 2.0460 - accuracy: 0.2857
Epoch 8/500
1/1 [==============================] - 0s 12ms/step - loss: 2.0395 - accuracy: 0.2857
Epoch 9/500
1/1 [==============================] - 0s 10ms/step - loss: 2.0330 - accuracy: 0.2857
Epoch 10/500
1/1 [==============================] - 0s 10ms/step - loss: 2.0263 - accuracy: 0.2857
Epoch 11/500
1

In [39]:
#predicting the next word using an initial sentence
input_phrase = "any sequence of your wish"
next_words = 5
  
for _ in range(next_words):
	token_list = tokenizer.texts_to_sequences([input_phrase])[0] #converting our input_phrase to tokens and excluding the out of vcabulary words
	token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre') #padding the input_phrase
	predicted = model.predict_classes(token_list, verbose=0) #predicting the token of the next word using our trained model
	output_word = "" #initialising output word as blank at the beginning
	for word, index in tokenizer.word_index.items():
		if index == predicted:
			output_word = word #converting the token back to the corresponding word and storing it in the output_word
			break
	input_phrase += " " + output_word
print(input_phrase)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


any sequence of your wish is my power power electrcity
